In [1]:
# Librairies
import pandas as pd
import numpy as np
import requests
import zipfile
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    GridSearchCV, cross_val_score,
    RandomizedSearchCV, cross_validate,
    StratifiedKFold
)
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

### Importation des données

In [8]:
data_path = r'C:\Users\nmako\OneDrive\Documents\Data-Science-Projects-Repository\Machine Learning TP\Application-ML-Detection-Defauts-Paiements-Credits-Bancaires\data\raw\credit_card_default.csv'
data = pd.read_csv(data_path)
df = data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          30000 non-null  int64 
 1   limit_bal                   30000 non-null  int64 
 2   sex                         30000 non-null  object
 3   education                   30000 non-null  object
 4   marriage                    30000 non-null  object
 5   age                         30000 non-null  int64 
 6   payment_status_sep          30000 non-null  int64 
 7   payment_status_aug          30000 non-null  int64 
 8   payment_status_jul          30000 non-null  int64 
 9   payment_status_jun          30000 non-null  int64 
 10  payment_status_may          30000 non-null  int64 
 11  payment_status_apr          30000 non-null  int64 
 12  bill_statement_sep          30000 non-null  int64 
 13  bill_statement_aug          30000 non-null  in